<h1 style="color:black" align="center">Семинар №9</h1>

<h1 style="color:#008B8B">1. Градиентный бустинг</h1>

На лекциях обсуждался градиентный бустинг - один из самых мощных методов в машинном обучении. Он позволяет воспользоваться выразительной силой решающих деревьев и при этом контролировать их переобучение. Ниже мы пройдём по основным блокам градиентного бустинга и поймём, почему они устроены именно так.

Для начала вспомним основные принципы градиентного бустинга. Будем искать алгоритм, оптимизирующий некоторую дифференцируемую функцию потерь $L(y, z)$, в виде взвешенной суммы базовых алгоритмов:

$\large a_N(x)
=
\sum\limits_{n = 0}^{N}
    \gamma_n b_n(x).$

Как правило, веса $\gamma_n$ не подбираются и полагаются равными единице (поскольку, как правило, в деревьях всё равно потом тщательно подбираются прогнозы в листьях), но пока будет рассматривать общий случай.

Идея бустинга заключается в последовательном построении алгоритмов, каждый из которых учитывает ошибки построенной до сих пор композиции:

$\large \sum\limits_{i = 1}^{\ell}
    L(y_i, a_{N - 1}(x_i) + \gamma_N b_N(x_i))
\to
\underset{b_N, \gamma_N}{\text{min}}$


После выбора каких-нибудь простых $\gamma_0$ и $b_0(x)$ (например, для задачи регрессии можно положить $\gamma_0 = 1$ и $b_0(x) = \frac 1\ell \sum\limits_{i=1}^\ell y_i$) все последующие базовые алгоритмы стараются приблизить антиградиент функционала ошибки, взятый в точках $z = a_{N - 1}(x_i)$:

$\large s_i
=
-
\left.
\frac{\partial L(y_i, z)}{\partial z}
\right|_{z = a_{N - 1}(x_i)}$


При этом приближается антиградиент с точки зрения квадратичной функции потерь:

$\large b_N(x)
=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \sum\limits_{i = 1}^{\ell}
        \left(
            b(x_i) - s_i
        \right)^2$


Подбор коэффициентов производится просто через задачу одномерной оптимизации:

$\large \gamma_N
=
\underset{\gamma \in \mathbb{R}}{\text{argmin}}
\sum\limits_{i=1}^{\ell}
L(y_i, a_{N-1}(x_i) + \gamma b_N (x_i))$


Обсудим теперь подробнее некоторые моменты.

<h2 style="color:#008B8B">1.1 Почему градиентный бустинг устроен именно так?</h2>

### Зачем сдвиги в бустинге считаются через производные функции потерь?

Почему нельзя использовать сдвиги вида $y_i - a_{N - 1}(x_i)$? Казалось бы, если удастся хорошо приблизить эти отклонения с помощью $b_N(x)$, то будет выполнено $a_{N-1}(x_i) + b_N(x_i) \approx y_i$.

Мы решаем сложные задачи, в которых, скорее всего, ещё и не идеальные данные, есть шумы, выбросы и так далее. Это означает, что не стоит рассчитывать на получение непереобученной модели с нулевой ошибкой на обучающей выборке. Мы это понимаем, и поэтому через функцию потерь определяем, какие ошибки более приемлемы, чем другие. Если использовать сдвиги, равные разности правильного ответа и прогноза текущей композиции, то мы полностью игнорируем эти требования. Разберём несколько примеров.

**Пример 1.** Начнём с регрессии. Допустим, у нас несимметричная функция потерь, где мы сильнее штрафуем за завышение прогноза:

$\large L(y, z)
=
\frac{1}{2}
(10 [z \geq y] + [z < y])
(y - z)^2$

Рассмотрим два объекта $x_1$ и $x_2$ с правильными ответами $y_1 = y_2 = 0$ и прогнозами текущей композиции $a_{N - 1}(x_1) = 5$ и $a_{N - 1}(x_2) = -5$. Если взять сдвиги, равные $y_i - a_{N - 1}(x_i)$, то базовая модель будет пытаться одинаково скорректировать прогнозы на обоих объектах. Но, конечно, надо сильнее сконцентрироваться на коррекции прогноза на первом объекте, поскольку на нём штраф больше: $L(0, 5) = 125 > 12.5 = L(0, -5)$. Сдвиги, посчитанные через частные производные, лучше отражают приоритеты: $s_1 = -50$, $s_2 = 5$. При этом не страшно, что на первом объекте мы сделаем слишком большую корректировку - это будет исправлено с помощью веса $\gamma_N$ базовой модели или с помощью других техник.

**Пример 2.** Обсудим теперь обучение на абсолютную ошибку $L(y, z) = |y - z|$.
Вычислим для неё сдвиги:

$\large s_i = - \left. \frac{\partial |y_i - z|}{\partial 
    z} \right|_{z=a_{N-1}(x_i)} = 
\Bigl. \text{sign}(y_i - z) \Bigr|_{z=a_{N-1}(x_i)} =
\text{sign}(y_i - a_{N-1}(x_i))$

И это снова более логично, чем обучение на $y_i - a_{N - 1}(x_i)$! Мы знаем, что с точки зрения квадратичной ошибки даже небольшое уменьшение серьёзной ошибки оказывается лучше, чем доведение небольшой ошибки до нуля:

$\large L(0, 1000) - L(0, 999) = 1999 > 1 = L(0, 1) - L(0, 0).$

Как мы видим, при уменьшении прогноза тысячи на еденицу уменьшает ошибку на 1999, что оказывается лучше, чем доведение небольшой ошибки до нуля, так как ошибка изменится всего на еденицу. 

Поэтому для квадратичной ошибки разумно использовать сдвиги $y_i - a_{N - 1}(x_i)$, поскольку чем больше отклонение от правильного ответа, тем больше штраф и тем сильнее надо корректировать прогнозы. У абсолютной ошибки такого свойства нет, изменения штрафа не зависят от отклонения прогноза от факта. Поэтому независимо от того, на каком объекте мы приблизим прогноз к правильному ответу на единицу, средняя абсолютная ошибка улучшится на одну и ту же величину. У нас как раз получилось, что сдвиги на всех объектах одинаковы по модулю и отличаются лишь знаком - это соответствует описанному выше свойству абсолютной ошибки.

**Пример 3.** Перейдём к классификации. Если у нас два класса ($\mathbb{Y} = \{-1, +1\}$), и сама композиция также возвращает прогноз из $\mathbb{Y}$, то отклонения всегда будут принимать значения из небольшого множества: $|y_i - a_{N - 1}(x_i)| \in \{0, +2\}$. 


Получается, что если модель уже выдаёт правильный ответ, то сдвиги будут равню нулю, и никакие модификации для этого объекта вноситься не будут. Разумеется, это нас не устраивает - как правило, функции потерь в классификации стараются максимизировать отступ (или хотя бы довести его до определённого положительного значения), а полученные нами сдвиги никак не зависят от отступа.

Чтобы это исправить, договоримся, что наша композиция $a_N(x)$ будет выдавать вещественные числа, и по смыслу они будут являться оценками *логита*, то есть логарифма отношения вероятности положительного класса к вероятности отрицательного класса:

$\large a_N(x)
=
\log
\frac{
    p(y = +1 | x)
}{
    1 - p(y = +1 | x)
}$

Если выразить отсюда вероятность, то получится

$\large p(y = +1 | x)
=
\sigma\left(
    a_N(x)
\right)
=
\frac{
    1
}{
    1 + \exp(-a_{N}(x))
}$

То есть сигмоида от модели будет оценивать вероятность положительного класса. Эту же идею мы использовали при обучении линейных классификаторов - там скалярное произведение как раз оценивало логиты. Получаем, что $a_N(x)$ - чем больше, тем выше уверенность в том, что объект положительный. С таким подходом обучение на $y_i - a_{N - 1}(x_i)$ выглядит ещё менее логичным, поскольку в этом случае мы будем пытаться уменьшить отступ там, где он большой положительный, то есть по сути будем запрещать модели быть уверенной в своём ответе. 

Например, имеется положительный равный $y_i = +1$ и построенна композиция выдает ответ на этом объекте $a_{N-1} (x_i) = 10000$, следовательно модель уверена в том, что этот объект положителен, у нее большой отступ на этом объекте. Отступ на этом объекте будет равен $y_i - a_{N-1} = -9999$, сдвиг говорит о том, что необходимо уменьшать прогноз модели, так как выход модели должен быть равен $+1$, а мы выдали $10000$. Следовательно, если использовать данные сдвиги, они будут запрещать строить уверенные модели, если модель уверена на некотором объекте, сдвиги будут уменьшать отступ объекта.

В завершение посчитаем сдвиги для логистической функции потерь:
$L(y, z) = \log (1 + \exp(-yz))$. Для этого вспомним, что логистическая функция потерь выражается через сигмоиду $\sigma(x) = \frac{1}{1 + \exp(-x)}$ следующим образом:

$\large L(y, z) = \log \left( \frac{1}{\sigma(yz)} \right) = -\log \sigma(yz)$

Далее, пользуясь формулой для производной сигмоиды $\sigma'(x) = \sigma(x)(1 - \sigma(x))$, получаем

\begin{align*}
    s_i &= \left. \frac{\partial \log \sigma(y_i z)}{\partial z} 
        \right|_{z=a_{N-1}(x_i)} =
    \left. \frac{1}{\sigma(y_i z)} \sigma(y_i z) (1 - \sigma(y_i z)) y_i
        \right|_{z=a_{N-1}(x_i)} =\\
    &= (1-\sigma(y_i a_{N-1}(x_i))) y_i =
    \frac{y_i}{1 + \exp(y_i a_{N-1}(x_i))}.
\end{align*}

Заметим, что чем больше отступ, тем меньше будут сдвиги по модулю. Здесь числитель говорит в какую сторону необходимо двигаться, для положитльных объектов числитель и знаменатель будет положителен, поэтому модель будет стремиться увеличивать прогноз композиции, для отрицательных объектов данная дробь будет отрицательной, поэтому прогноз композиции будет уменьшаться в сторону минус бесконечности. А знаметель говорит о том, чем больше отступ, тем меньше будет $|s_i|$. То есть, знаменатель говорит о том, что если на объекте уже хороший отступ, тогда сильно нет необходимости что-то изменять.